In [ ]:
# Install necessary dependencies
!pip install -U -q PyYAML
!pip install -U -q tensorboard
!pip install -U -q torch torchvision torchaudio
!pip install -U -q opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.17.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 33.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/compiled_data'

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class SimpleYOLO(nn.Module):
    def __init__(self, num_classes):
        super(SimpleYOLO, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

# Number of classes
num_classes = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SimpleYOLO(num_classes=num_classes).to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

# Define transformations for the training, validation, and testing sets
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

# Load the datasets
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/compiled_data/train', transform=transform)
val_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/compiled_data/val', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/compiled_data/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Verify the class names
class_names = train_dataset.classes
print(f"Class names: {class_names}")


Class names: ['NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")


Epoch 1, Loss: 0.19749788145629177
Epoch 2, Loss: 0.15265044663390664
Epoch 3, Loss: 0.11882755273131665
Epoch 4, Loss: 0.12271478937338173
Epoch 5, Loss: 0.10535326139776298
Epoch 6, Loss: 0.1020481925940731
Epoch 7, Loss: 0.09995883225326428
Epoch 8, Loss: 0.08983744247927375
Epoch 9, Loss: 0.0770152740970574
Epoch 10, Loss: 0.12573980015981412
Epoch 11, Loss: 0.07546224769259809


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


In [ ]:
import matplotlib.pyplot as plt

model.eval()
sample_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
images, labels = next(iter(sample_loader))
images = images.to(device)
outputs = model(images)
_, predicted = torch.max(outputs.data, 1)

# Visualize the image and prediction
image = images[0].cpu().numpy().transpose(1, 2, 0)
plt.imshow(image)
plt.title(f"Predicted: {classes[predicted[0]]}, Actual: {classes[labels[0]]}")
plt.show()
